# Pricing American Options

Pricing American options is more difficult because the excercise time
must be computed simultaneously with the payoff of a particular path
Longstaff and Schwartz proposed a method based on linear regression for
pricing American options.  

Since the price of an American call option is the same as a European
call, we only deal with American put options.

In [1]:
# Import necessary packages
import qmcpy as qp
import numpy as np
import time

#### Initialization
First we set up the basic common praramters for our examples.

In [2]:
initPrice = 100 # initial stock price
interest = 0.05 # risk-free interest rate
vol = 0.5 # volatility
callput = 'put' # put options
strike = 130 # strike price
tfinal = 1/4 # mature time
d = 13 # weekly monitoring
absTol = 0.05 # absolute tolerance of a nickel
relTol = 0 # zero relative tolerance
sampleSize = 10**6 # number of smaple size

In [3]:
AmericanPut = qp.AmericanOption(qp.IIDStdUniform(dimension=d), volatility= vol,start_price= initPrice,
                       strike_price=strike, interest_rate = interest,t_final=tfinal,call_put=callput)
x = AmericanPut.discrete_distrib.gen_samples(sampleSize)
y = AmericanPut.f(x)
print(" After generating ", f"{int(sampleSize):,}", "iid paths, the estimation of the fair price is",f"{y.mean():.4f}")

 After generating  1,000,000 iid paths, the estimation of the fair price is 31.2664


In [6]:
start = time.time()
sc = qp.CubMCCLT(AmericanPut,absTol)
solution, data = sc.integrate()
end = time.time()
print("To reach the absolute error tolerance = ",absTol, ", we need to generate ", f"{int(data.n_total):,}",
      "iid paths, the estimation of the fair price is",f"{solution:.4f}")
print("The total computing time is {}".format(end-start))

To reach the absolute error tolerance =  0.05 , we need to generate  1,435,985 iid paths, the estimation of the fair price is 31.2427
The total computing time is 4.932486534118652


If we use Quasi-Monte Carlo method, you can see the improvement of the number of paths and computing time.

In [9]:
AmericanPut = qp.AmericanOption(qp.DigitalNetB2(dimension=d), volatility= vol,start_price= initPrice,
                       strike_price=strike, interest_rate = interest,t_final=tfinal,call_put=callput)
start = time.time()
sampleSize = 2**12
x = AmericanPut.discrete_distrib.gen_samples(sampleSize)
y = AmericanPut.f(x)
end = time.time()
print(" After generating ", f"{int(sampleSize):,}", "paths, the estimation of the fair price is",f"{y.mean():.4f}")
print("The total computing time is {}".format(end-start))

 After generating  4,096 paths, the estimation of the fair price is 31.1745
The total computing time is 0.014037609100341797


In [8]:
EuroCall = qp.EuropeanOption(qp.IIDStdUniform(dimension=d,seed=7), volatility= vol,start_price= initPrice,
                       strike_price=strike, interest_rate = interest,t_final=tfinal,call_put=callput)
print("The exact price of this European Call Option is ",f"{EuroCall.get_exact_value():.4f}")

The exact price of this European Call Option is  30.7359


Notice that these two prices are similar.  If the interest rate is
decreased, then the prices are even closer together.